In [6]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [7]:
# Sample DataFrame
database = pd.read_csv('EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv')

database.head()

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,297836.0831
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,484945.1466
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,106705.9281
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,481828.4303
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,687684.7752


In [8]:
# Create a list with existing columns on database
column_list = database.columns.tolist()

print(column_list)

['OBJECTID', 'GEOID10', 'GEOID20', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'CSA', 'CSA_Name', 'CBSA', 'CBSA_Name', 'CBSA_POP', 'CBSA_EMP', 'CBSA_WRK', 'Ac_Total', 'Ac_Water', 'Ac_Land', 'Ac_Unpr', 'TotPop', 'CountHU', 'HH', 'P_WrkAge', 'AutoOwn0', 'Pct_AO0', 'AutoOwn1', 'Pct_AO1', 'AutoOwn2p', 'Pct_AO2p', 'Workers', 'R_LowWageWk', 'R_MedWageWk', 'R_HiWageWk', 'R_PCTLOWWAGE', 'TotEmp', 'E5_Ret', 'E5_Off', 'E5_Ind', 'E5_Svc', 'E5_Ent', 'E8_Ret', 'E8_off', 'E8_Ind', 'E8_Svc', 'E8_Ent', 'E8_Ed', 'E8_Hlth', 'E8_Pub', 'E_LowWageWk', 'E_MedWageWk', 'E_HiWageWk', 'E_PctLowWage', 'D1A', 'D1B', 'D1C', 'D1C5_RET', 'D1C5_OFF', 'D1C5_IND', 'D1C5_SVC', 'D1C5_ENT', 'D1C8_RET', 'D1C8_OFF', 'D1C8_IND', 'D1C8_SVC', 'D1C8_ENT', 'D1C8_ED', 'D1C8_HLTH', 'D1C8_PUB', 'D1D', 'D1_FLAG', 'D2A_JPHH', 'D2B_E5MIX', 'D2B_E5MIXA', 'D2B_E8MIX', 'D2B_E8MIXA', 'D2A_EPHHM', 'D2C_TRPMX1', 'D2C_TRPMX2', 'D2C_TRIPEQ', 'D2R_JOBPOP', 'D2R_WRKEMP', 'D2A_WRKEMP', 'D2C_WREMLX', 'D3A', 'D3AAO', 'D3AMM', 'D3APO', 'D3B', 'D3

In [9]:
# Dropping columns
columns_to_drop = ['COUNTYFP','STATEFP', 'CSA', 'CSA_Name', 'OBJECTID', 'GEOID10', 'GEOID20', 'TRACTCE', 'BLKGRPCE', 'Ac_Water', 'Ac_Land'
                   , 'TotPop', 'CountHU', 'HH', 'Pct_AO0','Pct_AO1','Pct_AO2p','R_PCTLOWWAGE'
                   ,'E5_Ret', 'E5_Off', 'E5_Ind', 'E5_Svc', 'E5_Ent', 'E8_Ret', 'E8_off'
                   , 'E8_Ind', 'E8_Svc', 'E8_Ent', 'E8_Ed', 'E8_Hlth', 'E8_Pub', 'E_LowWageWk'
                   , 'E_MedWageWk', 'E_HiWageWk', 'E_PctLowWage', 'D1A', 'D1C5_RET', 'D1C5_OFF'
                   , 'D1C5_IND', 'D1C5_SVC', 'D1C5_ENT', 'D1C8_RET', 'D1C8_OFF', 'D1C8_IND', 'D1C8_SVC'
                   , 'D1C8_ENT', 'D1C8_ED', 'D1C8_HLTH', 'D1C8_PUB', 'D1D', 'D1_FLAG', 'D2A_JPHH'
                   , 'D2B_E5MIX', 'D2B_E5MIXA', 'D2B_E8MIX', 'D2B_E8MIXA', 'D2A_EPHHM', 'D2C_TRPMX1'
                   , 'D2C_TRPMX2', 'D2C_TRIPEQ', 'D2R_JOBPOP', 'D2R_WRKEMP', 'D2A_WRKEMP', 'D2C_WREMLX'
                   , 'D3A', 'D3AAO', 'D3AMM', 'D3APO', 'D3B', 'D3BAO', 'D3BMM3', 'D3BMM4', 'D3BPO3', 'D3BPO4'
                   , 'D4B025', 'D4B050', 'D4C', 'D4D', 'D4E', 'D5AE', 'D5BR', 'D5BE', 'D5CR', 'D5CRI'
                   , 'D5CE', 'D5CEI', 'D5DR', 'D5DRI', 'D5DE', 'D5DEI', 'D2A_Ranked', 'D2B_Ranked'
                   , 'D3B_Ranked', 'D4A_Ranked', 'Shape_Length', 'Shape_Area']

database = database.drop(columns=columns_to_drop)

database.head()

,CBSA,CBSA_Name,CBSA_POP,CBSA_EMP,CBSA_WRK,Ac_Total,Ac_Unpr,P_WrkAge,AutoOwn0,AutoOwn1,...,Workers,R_LowWageWk,R_MedWageWk,R_HiWageWk,TotEmp,D1B,D1C,D4A,D5AR,NatWalkInd
0,19100.0,"Dallas-Fort Worth-Arlington, TX",7189384,3545715,3364458,73.595028,73.595028,0.549,69,39,...,412,99,122,191,66,16.332625,0.896800,362.10,433601,14.000000
1,19100.0,"Dallas-Fort Worth-Arlington, TX",7189384,3545715,3364458,119.829909,119.214200,0.466,0,168,...,395,76,107,212,25,5.955666,0.209707,718.84,386504,10.833333
2,19100.0,"Dallas-Fort Worth-Arlington, TX",7189384,3545715,3364458,26.367053,26.367050,0.811,19,143,...,463,136,189,138,0,27.951553,0.000000,398.31,404573,8.333333
3,19100.0,"Dallas-Fort Worth-Arlington, TX",7189384,3545715,3364458,119.060687,119.060687,0.638,0,43,...,431,60,69,302,253,7.592767,2.124967,386.24,423099,15.666667
4,19100.0,"Dallas-Fort Worth-Arlington, TX",7189384,3545715,3364458,169.927211,148.742920,0.506,5,67,...,579,91,84,404,32,6.373413,0.215136,638.37,335700,10.166667


In [ ]:
new_column_list = database.columns.tolist()

print(new_column_list)

In [ ]:
# Specify the columns on which you want to check for missing values
columns_check_na = ['CBSA', 'CBSA_Name']

# Drop rows with missing values in the specified columns
database_cleaned = database.dropna(subset=columns_to_check)

database_cleaned

In [ ]:
# Drop columns with value = 0
columns_check_zero = ['CBSA_EMP','CBSA_WRK']

database_cleaned = database_cleaned[(database_cleaned[columns_to_check] != 0).all(axis=1)]

database_cleaned

In [ ]:
# Filter database to only show results from NY state (NY)
# Specify the column and the value you want to filter
column_to_filter = 'CBSA_Name'
value_to_match = ', NY'

# Use boolean indexing to filter the DataFrame
filtered_database = database_cleaned[database_cleaned[column_to_filter].str.contains(value_to_match, na=False)]

filtered_database = filtered_database.reset_index(drop=True)
filtered_database

In [ ]:
# Assuming 'your_dataset' is your DataFrame
filtered_database['Perc_Local_wrk'] = (filtered_database['CBSA_WRK'] / filtered_database['CBSA_POP'] * 100).round(2)

# Find the index of the 'CBSA_WRK' column
wrk_column_index = filtered_database.columns.get_loc('CBSA_WRK')

# Insert the 'Perc_Local_wrk' column after the 'CBSA_WRK' column
filtered_database.insert(wrk_column_index + 1, 'Perc_Local_wrk', filtered_database.pop('Perc_Local_wrk'))

filtered_database

In [ ]:
# Getting unique values in the 'Perc_Local_wrk' column as list
unique_Perc_Local_wrk =[filtered_database['Perc_Local_wrk'].unique()]

print(f"Number of unique CBSA Names: {filtered_database['Perc_Local_wrk'].nunique()}")

# Display unique values
unique_Perc_Local_wrk

In [ ]:
# Getting unique values in the 'CBSA_Name' column as list
unique_values =[filtered_database['CBSA_Name'].unique()]

print(f"Number of unique CBSA Names: {filtered_database['CBSA_Name'].nunique()}")

# Display unique values
unique_values

In [5]:
# Saving filtered_database to CSV file to future work
# Specify the path where you want to save the CSV file
csv_file_path = 'filtered_database.csv'

# Save the DataFrame to a CSV file
filtered_database.to_csv(csv_file_path, index=False)

NameError: name 'filtered_database' is not defined

In [ ]:
# Openning a clean_database dataframe to project continuity

clean_database = pd.read_csv('filtered_database.csv')

clean_database.head()